# 3. Budowa modeli

In [1]:
#sciagniecie wszystkich wykorzystancyh bibliotek
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import statistics
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
#sciagniecie danych wykorzystanych do budowy modelu
import pickle
pick_read = open('/content/gdrive/My Drive/X_train.pkl','rb')
X_train = pickle.load(pick_read)
pick_read.close()
pick_read = open('gdrive/My Drive/X_val.pkl','rb')
X_val = pickle.load(pick_read)
pick_read.close()
pick_read = open('gdrive/My Drive/X_test.pkl','rb')
X_test = pickle.load(pick_read)
pick_read.close()

pick_read = open('gdrive/My Drive/y_train.pkl','rb')
y_train = pickle.load(pick_read)
pick_read.close()
pick_read = open('gdrive/My Drive/y_val.pkl','rb')
y_val = pickle.load(pick_read)
pick_read.close()
pick_read = open('gdrive/My Drive/y_test.pkl','rb')
y_test = pickle.load(pick_read)
pick_read.close()

In [ ]:
# utworzenie funkcji, ktora zostala wykorzystana do autmatyzacji procesu 
#budowy architektury splotowych sieci neuronowych 
#Dana funkcja posiada wiele parametrow, ktore sluza do zmiany hiperparametrow modelu
# conv_amount - ile model powinien miec warstw splotowych
# activation - jaka funkcja aktywacji powinna zostac zastosowana
# input_shape - jakie wymiary maja dane wejsciowe
# kernel_conv_amount - ile filtrow powinny posiadac poszczegolne warstwy splotowe zbudowanego modelu
# kernel_conv_dim - jakie pole recepcyjne powinny posiadac warstwy splotowe
# kernel_pool_dim - jakie pole recepcyjne powinny posiadac warstwy laczace
def CNN_builder(conv_amount,
                activation, 
                input_shape, 
                kernel_conv_amount, 
                kernel_conv_dim,
                kernel_pool_dim):
    
    
    model=tf.keras.Sequential()
    model.add(Conv2D(kernel_conv_amount[0],kernel_conv_dim[0],activation=activation,input_shape=(input_shape)))
    for i in range(1,conv_amount):
        model.add(Conv2D(kernel_conv_amount[i],kernel_conv_dim[i],activation=activation,padding='SAME'))
        model.add(MaxPooling2D(kernel_pool_dim, padding='SAME'))
    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(50, activation='softmax'))
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    return model

In [ ]:
#utworzenie modeli
model1=CNN_builder(4,'relu',(200,200,1),[16,32,64,128],[(3,3),(3,3),(3,3),(3,3)],(2,2))

model2=CNN_builder(5,'relu',(200,200,1),[16,32,64,128,128],[(3,3),(3,3),(3,3),(3,3),(3,3)],(3,3))

model3=CNN_builder(6,'relu',(200,200,1),[16,32,45,64,128,256],[(3,3),(3,3),(3,3),(3,3),(3,3),(3,3)],(2,2))              

model4=CNN_builder(8,'relu',(200,200,1),[16,16,32,64,128,128,256,256],[(3,3),(3,3),(3,3),(3,3),(3,3),(3,3),(3,3),(3,3)],(2,2))              

model5=CNN_builder(7,'relu',(200,200,1),[16,32,45,64,128,128,256],[(3,3),(3,3),(3,3),(3,3),(3,3),(3,3),(3,3)],(2,2))              

model6=CNN_builder(9,'relu',(200,200,1),[16,16,32,32,64,128,128,256,256],[(3,3),(3,3),(3,3),(3,3),(3,3),(3,3),(3,3),(3,3),(3,3)],(2,2))              


In [ ]:
#utworzenie metody early stopping
early_stop = EarlyStopping(monitor='val_loss',
                           min_delta=0, 
                           patience=10, 
                           verbose = 1, 
                           mode='min', 
                           restore_best_weights = True)

In [ ]:
# utworzenie petli, ktora zostala wykorzystana do trenowania zbudowanych modeli
histories=[]
for model in [model1,model2,model3,model4,model5,model6]:
    tf.debugging.set_log_device_placement(True)

    history=model.fit(
              X_train, 
              y_train, 
              batch_size = 32, 
              epochs = 100, 
              validation_data=(X_val, y_val), 
              callbacks=[early_stop])
    histories.append(history)

Epoch 1/100
544/544 [==============================] - 7s 12ms/step - loss: 3.2638 - accuracy: 0.1176 - val_loss: 3.1157 - val_accuracy: 0.1669
Epoch 2/100
544/544 [==============================] - 7s 12ms/step - loss: 3.0707 - accuracy: 0.1550 - val_loss: 2.9772 - val_accuracy: 0.1922
Epoch 3/100
544/544 [==============================] - 7s 12ms/step - loss: 2.8569 - accuracy: 0.1992 - val_loss: 2.8115 - val_accuracy: 0.2206
Epoch 4/100
544/544 [==============================] - 7s 12ms/step - loss: 2.6248 - accuracy: 0.2506 - val_loss: 2.7579 - val_accuracy: 0.2528
Epoch 5/100
544/544 [==============================] - 7s 12ms/step - loss: 2.3751 - accuracy: 0.2992 - val_loss: 2.7886 - val_accuracy: 0.2546
Epoch 6/100
544/544 [==============================] - 7s 12ms/step - loss: 2.1903 - accuracy: 0.3390 - val_loss: 2.8682 - val_accuracy: 0.2423
Epoch 7/100
544/544 [==============================] - 7s 12ms/step - loss: 2.0096 - accuracy: 0.3860 - val_loss: 2.9512 - val_accuracy:

In [ ]:
# zamieszczenie historii budowy modeli do pliku pickle
pick_insert = open('gdrive/My Drive/histories.pkl','wb')
pickle.dump(histories, pick_insert)
pick_insert.close()

models=[model1,model2,model3,model4,model5,model6]

In [ ]:
# zamieszczenie modeli do pliku pickle
pick_insert = open('gdrive/My Drive/models.pkl','wb')
pickle.dump(models, pick_insert)
pick_insert.close()